In [343]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [344]:
def predprod(DF):
  data_1h = np.zeros((len(DF['<CLOSE>']), 2))
  data_1h[:,1] = DF['<CLOSE>']

  for i in range(len(data_1h)):
    data_1h[i][0] = i

  # data_1h = np.zeros((len(data_15)//4, 2))

  # for i in range(len(data_1h)):
  #   data_1h[i][0] = i

  # for i in range(len(data_15)//4):
  #   data_1h[i,1] = data_15[i*4,1]

  return(data_1h)

In [345]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

In [346]:
def lin_regression_1h_funk(X, Y):
  lin_regression_1h = linear_model.LinearRegression()
  lin_regression_1h.fit(X, Y)
  linear_pred_1h = lin_regression_1h.predict(X)
  return(linear_pred_1h)

In [347]:
def linear_model_vkr(data_1h):

  lin_regression_1h = linear_model.LinearRegression()
  lin_regression_1h.fit(data_1h[-5:-1,0].reshape(-1, 1), data_1h[-5:-1,1])

  linear_predict_1h = np.zeros(len(data_1h[:,0])-4)
  for i in range(0, len(data_1h[:,0])-4):
    linear_predict_1h[i] = lin_regression_1h_funk(data_1h[i:i+4,0].reshape(-1, 1), data_1h[i:i+4,1])[-1]

  finally_forecast_linear_1h = lin_regression_1h.predict(data_1h[len(data_1h[:,0])-1:,0].reshape(-1, 1))

  return(finally_forecast_linear_1h, linear_predict_1h)

In [348]:
import warnings
import itertools

import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [349]:
def ARIMA_model(data_1h):

  #data_all = np.zeros(len(data_1h[:,1]))
  data_all = data_1h[:,1]

  warnings.filterwarnings("ignore")
  #del best_pdq
  p = range(0,4)
  d = q = range(0,2)
  pdq = list(itertools.product(p, d, q))
  best_pdq = (0,0,0)
  best_aic = np.inf

  for params in pdq:
    model_test = sm.tsa.arima.ARIMA(data_all, order = params)
    result_test = model_test.fit()
    if result_test.aic < best_aic:
      best_pdq = params
      best_aic = result_test.aic
  #print(best_pdq, best_aic)
  del best_aic
  del params

  ARIMA_model_1h = sm.tsa.arima.ARIMA(data_1h[:,1], order =(best_pdq))
  del best_pdq

  ARIMA_1h = ARIMA_model_1h.fit()

  ARIMA_1h_forecast = ARIMA_1h.forecast()

  ARIMA_1h_predict = np.delete(ARIMA_1h.predict(dynamic = False),[0,1,2,3], 0)

  return(ARIMA_1h_forecast, ARIMA_1h_predict)




In [350]:
!pip install arch
from arch import arch_model

In [351]:
def GARCH_model(data_1h):

  data_1h_incr = np.zeros(len(data_1h)-1)
  for i in range(0,len(data_1h)-1):
    data_1h_incr[i] = (data_1h[i+1,1] - data_1h[i,1])/data_1h[i,1]

  am_SGARCH_1h = arch_model(data_1h_incr, p=1, o=0, q=1,vol='GARCH')
  res_SGARCH_1h = am_SGARCH_1h.fit(update_freq=5, disp='off')

  forecasts_SGARCH_1h = res_SGARCH_1h.forecast(horizon=1, start = 0, align = 'target', method='simulation')
  variance_SGARCH_1h = forecasts_SGARCH_1h.residual_variance

  predictions_SGARCH_1h = np.zeros(len(data_1h)-3)
  for i in range(0,len(data_1h)-3):
    predictions_SGARCH_1h[i] = data_1h[i+3,1] + data_1h[i+3,1]*variance_SGARCH_1h['h.1'][i+2]*np.random.normal(0,1)

  return(predictions_SGARCH_1h)



In [352]:
def MSE(data_1h):

  finally_forecast_linear_1h, linear_predict_1h = linear_model_vkr(data_1h)
  MSE_1h_linear = mean_squared_error(data_1h[4:,1], linear_predict_1h)

  ARIMA_1h_forecast, ARIMA_1h_predict = ARIMA_model(data_1h)
  MSE_1h_ARIMA = mean_squared_error(data_1h[4:,1], ARIMA_1h_predict)

  MSE_1h_sGARCH = mean_squared_error(data_1h[4:,1], GARCH_model(data_1h)[:-1])

  return(MSE_1h_linear, MSE_1h_ARIMA, MSE_1h_sGARCH)

In [353]:
def finally_forecast_1h_fun(data_1h):

  MSE_1h_linear, MSE_1h_ARIMA, MSE_1h_sGARCH = MSE(data_1h)

  finally_forecast_linear_1h, linear_predict_1h = linear_model_vkr(data_1h)

  ARIMA_1h_forecast, ARIMA_1h_predict = ARIMA_model(data_1h)


  finally_forecast_1h = finally_forecast_linear_1h * MSE_1h_linear/(MSE_1h_linear + MSE_1h_ARIMA+ MSE_1h_sGARCH) + ARIMA_1h_forecast * MSE_1h_ARIMA/(MSE_1h_linear + MSE_1h_ARIMA+ MSE_1h_sGARCH) + GARCH_model(data_1h)[-1] * MSE_1h_sGARCH/(MSE_1h_linear + MSE_1h_ARIMA+ MSE_1h_sGARCH)

  return(finally_forecast_1h)

In [354]:
DF = pd.read_csv('/content/XLTCZUSD_231001_231201.csv', sep=';')
#DF.head()
pred_result = []

for data in range(0, len(DF["<CLOSE>"]) - 988):
  data_1h = predprod(DF.iloc[data:data+263])
  pred_result = np.append(pred_result, finally_forecast_1h_fun(data_1h))
  if (data % 50) == 0:
    print('iteration ', data, 'successfully\n')

/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


iteration  0 successfully



/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/uni

iteration  50 successfully



/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/uni

iteration  100 successfully

iteration  150 successfully

iteration  200 successfully

iteration  250 successfully



/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/uni

iteration  300 successfully

iteration  350 successfully

iteration  400 successfully

iteration  450 successfully



In [367]:
#LTC 500
# pred_result = [  61.44038221,   61.50344085,   56.4541288 ,   52.87937598,
#          61.46302673,   61.2795573 ,   61.01901438,   61.10729356,
#          58.24896462,   66.74943957,   61.34649762,   61.27949351,
#          61.21299592,   61.19557652,   60.85754793,   60.76502427,
#          60.95692289,   61.12674645,   61.13821498,   61.16486746,
#          61.02849775,   61.0098109 ,   61.18971363,   61.16456737,
#          60.99125196,   61.04239599,   60.88645487,   56.66841896,
#          61.16513852,   61.27282904,   61.29907316,   61.24260569,
#          61.19568985,   61.23613731,  335.83777031,   61.34351354,
#          61.42446217,   61.41878732,   61.3400568 ,   61.37145564,
#          61.42680358,   61.6597885 ,   61.73999219,   61.57577162,
#          61.4063634 ,   61.38951571,   61.34829648,   61.34452619,
#          61.2920199 ,   61.5262011 ,   61.77275378,   61.6707702 ,
#          61.62222501,   61.59288959,   61.64057925,   61.70289039,
#          61.68811886,   61.61145576,   61.56788102,   61.50481741,
#          61.53016083,   61.4816766 ,   61.51297008,   61.57105852,
#          61.56218931,   61.56054156,   61.56890792,   61.52452315,
#          61.46075189,   61.88580014,  363.05342049,   61.89913381,
#          38.99795755,   61.86255127,   61.79949521,   61.68969934,
#          61.60103061,   61.47607559,   61.66107566,   61.74357218,
#          61.68609369,   61.63553364,   61.5347568 ,   61.53881164,
#          61.55571768,   61.56924101,   61.56821439,   61.57843805,
#          61.53244845,   61.49367056,   61.49033133,   61.44908968,
#          61.5758927 ,   61.7240782 ,   61.81180455,   61.94737337,
#          61.90811968,   62.01273524,   61.87974227,   61.66702449,
#          61.60846273,   61.64565298,   61.83006639,   61.85007749,
#          61.8390164 ,   61.76348656,   62.36468745,   63.04329661,
#          63.28566858,   63.2940267 ,   63.03260677,   62.91105245,
#          62.88918709,   63.49854044,   63.81717457,   64.09149596,
#          63.87410313,   63.63796755,   63.71832159,   63.97252962,
#          64.14632299,   63.99869377,   63.44595592,   63.31447836,
#          63.16422312,   62.96794906,   63.10065305,   63.10801409,
#          63.08976907,   63.16729453,   63.1417169 ,   63.16740716,
#          63.29185067,   63.19362807,   63.169054  ,   62.9708074 ,
#          62.88454847,   62.58091419,   62.18518294,   62.13677853,
#          62.19869028,   62.28906473,   62.17493722,   61.76436512,
#          61.75352045,   61.82341594,   62.00330113,   62.16655376,
#          62.15164449,   62.12476792,   62.04481707,   62.03055853,
#          62.12063812,   62.3969917 ,   62.5344023 ,   62.44853774,
#          62.18133444,   62.05725201,   61.84027785,   61.94344952,
#          61.92616075,   61.85247186,   61.76559284,   61.54646535,
#          61.45182388,   61.43825959,   61.47617465,   61.40214762,
#          61.03951057,   60.82512562,   60.76882483,   60.71460929,
#          60.42255626,   60.36230431,   60.19508283,   60.22700948,
#          60.39704797,   60.43625701,   60.57702   ,   60.5901257 ,
#          60.78169591,   60.79714784,   60.83097232,   61.01081166,
#          61.07241384,   61.21718789,   61.19171813,   61.32081332,
#          61.50929112,   61.49166638,   61.33791516,   61.21447022,
#          61.29016459,   61.50379184,   61.53524176,   61.61422554,
#          61.72746249,   61.75078792,   61.78614204,   62.09279601,
#          62.5574733 ,   62.71163688,   62.63104282,   62.78943064,
#          62.85722046,   63.47333592,   64.05900403,   64.02001696,
#          63.78155646,   63.55926269,   63.56760468,   63.65907437,
#          63.60045809,   63.35380866,   63.13251826,   63.18143228,
#          63.38876025,   63.50990821,   63.55449339,   63.52860615,
#          63.41891374,   63.4417932 ,   63.48288728,   63.65074805,
#          63.87914217,   63.9011213 ,   63.7963312 ,   63.9092529 ,
#          64.05270938,   64.13924644,   64.18603416,   64.1191723 ,
#          64.25434235,   64.32736568,   64.65958912,   65.03605837,
#          65.09531721,   64.83252902,   64.9602157 ,   64.98700693,
#          64.98088496,   64.88213074,   64.77259184,   64.80581462,
#          64.77106766,   64.90475632,   64.84137965,   64.91353458,
#          64.95447586,   64.93906733,   65.10317438,   65.05676704,
#          64.52500423,   64.12046232,   63.79725675,   63.84135115,
#          63.94490219,   64.05162904,   64.18071473,   64.03460588,
#          64.50238234,   64.92652446,   65.46309966,   65.22695592,
#          64.77558862,   64.4980008 ,   64.09920326,   64.62946027,
#          65.01815481,   65.36584155,   65.73501585,   65.92898469,
#          66.31213743,   66.43914054,   66.63461259,   66.55415733,
#          66.30229731,   66.52853415,   66.19056036,   66.34747508,
#          66.44593313,   66.35158287,   66.09526031,   65.97281846,
#          65.99563547,   66.70435167,  119.48807332,   67.23430928,
#          67.32492605,   67.24782801,   67.505084  ,   68.92248383,
#          69.32337305,   69.62289129,   69.1560983 ,   71.39078885,
#          71.00531401,   70.61128014,   69.96822505,   68.73169371,
#          69.15924654,   69.0830177 ,   69.77486797,   69.94229064,
#          70.27251932,   70.62704034,   70.88009161,   70.75847927,
#          69.18088953,   68.21702361,   67.88224632,   67.9815719 ,
#          68.64990295,   68.29995865,   68.69253641,   68.84305284,
#          69.17678626,   69.640003  ,   70.18303416,   70.05712622,
#          69.85826717,   69.65878083,   69.32141779,   69.39078938,
#         126.15014425,   68.30350869,   68.50441131,   68.55770743,
#          68.90174036,   68.95265042,   69.26812731,   69.70352366,
#          70.09013473,   69.2629917 ,   68.86847264,   68.48882331,
#          68.50255134,   68.49962972,   68.38016924,   68.5074156 ,
#          68.67554197,   68.87597602,   69.1646765 ,   69.32199221,
#          69.86814992,  125.78294463,   69.33317067,   69.33138788,
#          69.6065049 ,   70.19466567,   70.34500522,   70.04316523,
#          69.53026426,   69.59148281,   69.42376314,   68.71807409,
#          68.12197288,   67.33826044,   67.66275008,   67.78811332,
#          68.07125581,   68.23716017,   68.22443017,   68.70788164,
#          68.80697373,   68.71338791,   67.89788433,   67.83833407,
#          67.92073412,   68.09755574,   68.20932642,   68.12840698,
#          67.70627016,   67.68780811,   67.79753803,   67.84473165,
#          67.99503388,   68.16439006,   68.45299183,   68.02244853,
#          67.95170691,   67.20002835,   66.55193726,   66.16994332,
#          66.13917031,   66.45364888,   66.6816642 ,   66.72266446,
#          66.89884792,   67.38845655,   67.71247672,   67.95208712,
#          72.54053988,   68.15883077,   68.12943116,   68.0183187 ,
#          67.98269391,   67.93858683,   67.95039906,   67.91881122,
#          67.97004049,   67.81299182,   67.94352964,   67.94945724,
#          68.02381256,   68.27311976,   68.25812606,   68.18852256,
#          67.81931514,   67.71143986,   67.80597341,   67.84388849,
#          67.85034102, -140.57975437,   67.50905876,   67.41792172,
#          67.26387787,   67.27994373,   67.40490972,   67.46846454,
#          67.70225592,  -57.91375117,   68.02090138,   67.97592815,
#          67.94978153,   68.25378022,   68.32420327,   68.61227288,
#          68.87325852,   68.76734671,   68.96170542,   68.93882479,
#          68.87986694,   68.96061355,   68.79478864,   68.9819239 ,
#          68.88952221,   68.8268787 ,   68.85357721,   68.48124658,
#          68.30552827,   68.23483151,   68.05559284,   68.11276966,
#          68.18155531,   68.47725783,   68.87003884,   69.10760101,
#          69.37120149,   69.08030656,   69.06921915,   69.12880851,
#          69.47309094,   69.48780826,   69.20727994,   68.8718463 ,
#          68.89121451,   69.0088375 ,   69.22980892,   69.19325704,
#          69.20024507,   69.46317198,   69.45810389,   69.28000045,
#          68.92449661,   68.77592921,   68.92633334,   68.9463686 ,
#          68.78079966,   69.20813601,   69.87408095,   69.94028615,
#          70.1549948 ,   69.7195946 ,   69.47593688,   69.67370507,
#          68.96096714,   68.84672327,   68.55142695,   68.49230907,
#          68.51848021,   68.6316315 ,   68.59866482,   68.67257818,
#          68.88382521,   68.84420565,   68.83532046,   68.57025459,
#          68.3124488 ,   68.17811335,   67.69018843,   67.64680941,
#          67.85425517,   67.99267178,   68.24742   ,   67.95084446,
#          67.90643901,   68.18479781,   67.95833469,   67.34330981]
# right_direction = 271
# wrong_direction = 229
# capital = 97.14308763308235
# +103
# -397

array([  61.44038221,   61.50344085,   56.4541288 ,   52.87937598,
         61.46302673,   61.2795573 ,   61.01901438,   61.10729356,
         58.24896462,   66.74943957,   61.34649762,   61.27949351,
         61.21299592,   61.19557652,   60.85754793,   60.76502427,
         60.95692289,   61.12674645,   61.13821498,   61.16486746,
         61.02849775,   61.0098109 ,   61.18971363,   61.16456737,
         60.99125196,   61.04239599,   60.88645487,   56.66841896,
         61.16513852,   61.27282904,   61.29907316,   61.24260569,
         61.19568985,   61.23613731,  335.83777031,   61.34351354,
         61.42446217,   61.41878732,   61.3400568 ,   61.37145564,
         61.42680358,   61.6597885 ,   61.73999219,   61.57577162,
         61.4063634 ,   61.38951571,   61.34829648,   61.34452619,
         61.2920199 ,   61.5262011 ,   61.77275378,   61.6707702 ,
         61.62222501,   61.59288959,   61.64057925,   61.70289039,
         61.68811886,   61.61145576,   61.56788102,   61.50481

In [355]:
#SBER
# pred_result = [273.78025774, 273.94958759, 274.13293506, 274.4239995 ,
#        274.53244515, 274.79366624, 274.28705554, 274.08350637,
#        273.999604  , 273.94117463, 273.92690829, 273.85045734,
#        273.93672313, 274.05623794, 274.06903145, 274.10246714,
#        274.09945256, 274.11530058, 273.70648286, 273.33859727,
#        273.19158797, 273.10294032, 273.33198992, 273.37906937,
#        273.43251348, 273.23734704, 273.32663513, 273.36822099,
#        273.38401425, 273.42782475, 273.43121984, 273.56913148,
#        274.55804   , 274.92259403, 274.9672134 , 275.09159679,
#        275.89877129, 276.61955042, 276.7509782 , 276.40323009,
#        275.84894555, 276.02848818, 276.15334327, 276.35919098,
#        276.5132926 , 276.67063273, 276.21179755, 275.68364425,
#        275.37297369, 275.1412932 , 275.36304219, 275.43204743,
#        275.63641979, 275.8822206 , 276.01606148, 275.97701173,
#        275.96587857, 275.94632306, 275.90968382, 275.52151596,
#        275.65557127, 275.65683111, 275.69239316, 278.07344506,
#        275.3962415 , 275.65663296, 275.4335337 , 275.6109827 ,
#        275.59404564, 275.50333593, 278.71391489, 274.7607163 ,
#        274.417094  , 274.3121642 , 274.57351021, 274.55649605,
#        274.88853454, 275.11182291, 275.07689392, 274.85933039,
#        274.55633613, 275.10483628, 275.70763108, 275.92295643,
#        275.97191594, 275.81516541, 221.86895599, 274.65352776,
#        275.49755746, 275.76335307, 275.84336634, 275.7283853 ,
#        275.90998451, 273.8130828 , 275.75450707, 275.67736788,
#        275.54903392, 250.98910156, 275.55024405, 236.12609057,
#        275.82407108, 275.86163072, 276.51066078, 276.78268398,
#        276.82987805, 276.32826609, 276.10262021, 276.20029288,
#        276.23303125, 275.88714894, 275.86096404, 275.88459074,
#        275.95650666, 275.93429947, 275.88372656, 275.93138812,
#        275.40675739, 274.91745637, 274.6133082 , 274.89822385,
#        275.24144378, 275.19542182, 275.08387213, 275.45442683,
#        275.67976316, 276.04531882, 276.1183546 , 276.1691207 ,
#        276.31156706, 276.15707659, 277.16047325, 278.49113142,
#        278.91717056, 279.04496006, 278.75643204, 278.24101587,
#        277.91582734, 277.88658447, 277.69414634, 277.91187706,
#        278.01730185, 278.04026355, 278.23196396, 278.09907196,
#        278.20888629, 278.05887962, 277.68615162, 277.44585255,
#        277.33913114, 277.693353  , 277.67061643, 277.62986647,
#        277.6911856 , 277.32999133, 277.13479668, 276.94137611,
#        276.80528911, 276.97517461, 276.2900967 , 275.83937624,
#        275.86951446, 275.85891273, 276.01864429, 275.1091431 ,
#        274.39462764, 274.3865739 , 274.81393802, 275.11874845,
#        275.02956389, 274.79005011, 274.72300039, 274.84860396,
#        274.45203743, 274.9207476 , 275.34017787, 275.29315817,
#        275.37132585, 275.45314439, 275.29626233, 275.3194829 ,
#        275.16071875, 274.9269394 , 274.95895921, 274.98922837,
#        274.98442061, 274.96063284, 274.89317748, 275.96567272,
#        276.70880738, 276.71905427, 276.68519636, 276.32073724,
#        276.51430832, 276.78853055, 276.34662299, 276.09156025,
#        275.85448993, 275.8455658 , 275.93154819, 275.90450721]
# right_direction = 100
# wrong_direction = 100
# capital = 96.87
# +104
# -96

In [356]:
# LTC
# pred_result = [69.89950013, 69.63290904, 69.50015893, 69.3712722 , 69.36709326,
#        69.42462599, 69.496436  , 69.70816071, 69.53256036, 69.34896542,
#        69.22987136, 69.20698599, 69.36583278, 69.2681528 , 69.33749845,
#        69.15425377, 68.95432738, 68.72219317, 68.68223134, 68.93676721,
#        69.18337597, 69.43037689, 69.40132104, 69.39674619, 69.40700784,
#        69.40968709, 69.42542127, 69.30748994, 69.48271159, 69.31850641,
#        69.32869066, 69.3212877 , 69.62683094, 69.94529542, 70.14037043,
#        69.87015977, 69.39431141, 68.9612113 , 68.49291913, 68.48067541,
#        68.80681053, 69.13517111, 69.3279953 , 69.60234533, 69.38233383,
#        67.72109921, 66.83979844, 66.53438071, 67.3598682 , 68.16459115,
#        68.52497566, 68.25682175, 67.80670264, 67.81663785, 68.09267648,
#        68.30206348, 68.68621976, 68.7297552 , 68.57763847, 68.74015176,
#        68.86757263, 68.89102426, 69.12021557, 69.95508423, 70.03310425,
#        70.32779751, 68.95143909, 69.09674014, 70.01871393, 70.39698578,
#        71.19237596, 70.81141402, 71.10539757, 71.26637508, 71.17590202,
#        71.50396773, 71.56153279, 71.69351198, 71.76410634, 71.89149959,
#        72.12840824, 72.24405531, 72.00120281, 71.37711902, 71.12047698,
#        70.94310051, 70.9801566 , 70.96447569, 70.89660507, 70.85281073,
#        70.95960947, 71.15320226, 71.27483057, 71.42075384, 71.37459487,
#        71.40998565, 71.36606746, 71.37639296, 71.31164545, 71.12005819,
#        71.2376487 , 71.1935281 , 71.2310012 , 71.366761  , 71.12251389,
#        71.06664594, 71.11563882, 71.16383241, 71.03074876, 70.78481661,
#        70.61776089, 70.64551059, 70.68497143, 71.15941937, 71.49044734,
#        72.03485673, 72.00059501, 72.30329608, 71.95753984, 71.89830814,
#        72.04790486, 71.40236336, 71.57275473, 71.78515784, 72.11986153,
#        71.91999481, 70.62159532, 69.35048218, 68.85712562, 69.02551263,
#        69.16484177, 69.02133843, 68.66331122, 68.38829187, 68.3175883 ,
#        68.21642957, 67.65547394, 67.58157302, 67.32915708, 67.32188408,
#        67.36307778, 67.52262843, 67.67889754, 67.96010525, 68.09278556,
#        68.00844292, 68.17648705, 68.09731514, 67.8428398 , 67.24685723,
#        65.68400069, 64.67147638, 64.22889906, 64.42980255, 64.41356635,
#        64.27593387, 64.50924961, 64.67411745, 65.05740913, 64.91771737,
#        64.72796708, 64.61735788, 64.51217594, 64.78657185, 65.21300389,
#        65.59043482, 65.39392461, 65.23810472, 64.95100428, 64.83189737,
#        65.11877029, 65.28050093, 65.39230432, 65.56865676, 65.67736867,
#        65.93633903, 65.90346396, 65.65169678, 65.26064361, 64.91569801,
#        65.15284308, 65.49016819, 65.85468291, 65.84744881, 65.76352079,
#        65.53788165, 65.52241357, 65.69571841, 65.92923557, 66.04812392,
#        65.7966228 , 65.60625358, 65.53158486, 65.47238777, 65.43402924,
#        65.57755715, 65.66746614, 65.64511686, 65.59218184, 65.57387911]
# rigth_direction = 109
# wrong_direction = 91
# capital = 99.02947790138974
# + 40
# - 160

In [357]:
#BCH
# pred_result = [255.77383685, 253.04165134, 250.95714197, 249.22832106,
#        249.21633986, 249.62492097, 249.48547396, 248.37273148,
#        247.64799004, 246.4391591 , 246.5331245 , 247.10941703,
#        247.99502615, 247.25765808, 246.60887944, 244.72054716,
#        242.86876194, 242.14125679, 241.91753078, 242.94402444,
#        243.10659096, 243.37137773, 243.7038571 , 244.14843035,
#        244.82188583, 244.46737384, 244.28334363, 242.83497452,
#        242.81888829, 242.07129636, 242.29441532, 242.7426075 ,
#        243.28930948, 243.85721185, 243.58753223, 242.98039871,
#        240.73441827, 240.45203184, 239.78133662, 240.39336172,
#        241.58903524, 241.90230389, 239.7401444 , 237.06545549,
#        235.67605496, 233.96803109, 236.19419846, 236.13563709,
#        236.90191041, 237.4988485 , 236.98858698, 236.51664624,
#        235.20605767, 234.20152426, 234.23146483, 234.96416782,
#        236.48646928, 236.91855843, 237.3724143 , 237.27022165,
#        237.19983572, 237.14746439, 236.82965881, 236.34728536,
#        235.08050809, 234.19692318, 229.54978178, 230.33033426,
#        232.23194155, 235.30090121, 237.05907303, 236.48571199,
#        235.83120918, 236.300755  , 235.98002629, 237.09434908,
#        236.80684574, 237.04883232, 236.93633231, 236.36480928,
#        236.42551143, 236.46718818, 236.07840866, 235.83971898,
#        235.6156847 , 236.14623014, 236.41880478, 237.57857937,
#        237.89662473, 238.48564339, 238.92306425, 239.65975549,
#        239.43460507, 240.00326821, 240.61939627, 241.14044982,
#        241.50008914, 240.82194968, 240.45186367, 239.0719983 ,
#        239.55872431, 239.12444298, 239.57716072, 239.91817791,
#        239.41034438, 239.72376668, 239.79762566, 241.231764  ,
#        241.00776785, 240.75485026, 240.14335408, 239.44645778,
#        239.65963798, 238.52674279, 239.20102562, 238.96203699,
#        237.84325072, 237.46700557, 235.0258859 , 234.37268351,
#        234.46811938, 233.97371878, 235.43282937, 235.99102209,
#        235.39660686, 233.72517951, 235.28559048, 235.26683912,
#        235.32186442, 233.67320716, 232.14067214, 232.90507912,
#        234.30838136, 234.88271487, 235.38445802, 235.11383501,
#        234.55994656, 235.07539975, 234.83836962, 234.67867483,
#        234.9968776 , 236.08104986, 235.74902203, 236.30172488,
#        235.12911176, 234.74857985, 234.72534608, 234.39979427,
#        233.26631525, 231.07516112, 226.70965823, 223.59441151,
#        222.57998664, 223.3853305 , 222.81108675, 223.35912411,
#        225.26639682, 227.46302089, 228.2284271 , 226.12864885,
#        224.34064215, 223.36024323, 225.31411578, 228.23931753,
#        229.76927386, 231.33789271, 232.74608849, 234.20452803,
#        233.32114753, 232.13706076, 231.76544643, 231.42809819,
#        233.53719423, 235.41626484, 236.31226727, 237.84041803,
#        237.12039654, 236.13800005, 235.18285723, 234.31588814,
#        234.74818608, 235.99445945, 236.84844388, 236.75534815,
#        236.0650308 , 235.24167331, 235.0549042 , 235.36929464,
#        236.20191898, 236.08339211, 235.50747334, 235.02448928,
#        234.90631225, 235.21425497, 235.1746949 , 235.91629924,
#        236.08266414, 236.45600361, 236.28814141, 236.33260552]
# right_direction = 100
# wrong_direction = 100
# capital = 96.87
# + 104
# - 96

In [358]:
#BCH 1000
# pred_result = [ 2.29793099e+02,  2.29586966e+02,  2.29232095e+02,  2.29743523e+02,
#         2.29808989e+02,  2.29680409e+02,  2.29377168e+02,  2.28138643e+02,
#         2.27941660e+02,  2.28459414e+02,  2.27955111e+02,  2.28075568e+02,
#         2.27173254e+02,  2.27312806e+02,  2.28081231e+02,  2.28805451e+02,
#         2.29473726e+02,  2.28889800e+02,  2.27975024e+02,  2.27104757e+02,
#         2.27550163e+02,  2.28725246e+02,  2.29203175e+02,  2.29535048e+02,
#         2.28826112e+02,  2.28792697e+02,  2.28266953e+02,  2.27084012e+02,
#         2.26034018e+02,  2.25893856e+02,  2.25181166e+02,  2.23604109e+02,
#         2.23372475e+02,  2.23828403e+02,  2.24261305e+02,  2.24922743e+02,
#         2.23895144e+02,  2.24168929e+02,  2.23201282e+02,  2.20941758e+02,
#         2.21402322e+02,  2.21039816e+02,  2.22840144e+02,  2.24679687e+02,
#         2.25018300e+02,  2.25678785e+02,  2.24991458e+02,  2.24476909e+02,
#         2.24447793e+02,  2.25056694e+02,  2.26637292e+02,  2.27311747e+02,
#         2.27958072e+02,  2.27828853e+02,  2.29090975e+02,  2.29461033e+02,
#         2.29333398e+02,  2.29063161e+02,  2.29046053e+02,  2.29734973e+02,
#         2.30058425e+02,  2.29594287e+02,  2.28545133e+02,  2.28363939e+02,
#         2.28265969e+02,  2.27990172e+02,  2.28152027e+02,  2.26904449e+02,
#         2.26883431e+02,  2.26450292e+02,  2.24218512e+02,  2.24130997e+02,
#         2.24119534e+02,  2.24439628e+02,  2.25568223e+02,  2.25050379e+02,
#         2.25138131e+02,  2.25514113e+02,  2.25066323e+02,  2.25980260e+02,
#         2.25279179e+02,  2.26757188e+02,  2.27585199e+02,  2.19613471e+02,
#         2.20749613e+02,  2.28698021e+02,  2.29176814e+02,  2.28968175e+02,
#         2.28937075e+02,  2.28820130e+02,  2.30937573e+02,  2.29293213e+02,
#         2.32478643e+02,  7.73506241e+03,  2.30925450e+02,  2.31869443e+02,
#         2.31167306e+02,  2.29462794e+02,  2.27782802e+02,  2.28086860e+02,
#         2.29084817e+02,  2.29728347e+02,  2.29803797e+02,  2.29604762e+02,
#         2.30007414e+02,  2.30399673e+02,  2.29696875e+02,  2.29941911e+02,
#        -1.66622397e+05,  6.03887800e+02,  2.31065629e+02,  2.31754965e+02,
#         2.32446758e+02,  2.33432676e+02,  2.33630403e+02,  2.31490329e+02,
#         2.30267544e+02,  2.29928721e+02,  2.30176468e+02,  1.25371285e+07,
#         2.30991615e+02,  2.32108319e+02,  2.33285704e+02,  2.34039716e+02,
#         2.33941494e+02,  2.33792815e+02,  2.34503702e+02,  2.33412520e+02,
#         2.33380589e+02,  2.33241448e+02,  2.33352602e+02,  2.33661639e+02,
#         2.32494738e+02,  2.33264862e+02,  2.37204448e+02,  2.38115839e+02,
#         2.38315394e+02,  2.36379130e+02,  2.35020751e+02,  2.35407083e+02,
#         2.35723947e+02,  2.37518777e+02,  2.37535984e+02,  2.38133135e+02,
#         2.37756491e+02,  2.37947154e+02,  2.38146861e+02,  2.37905072e+02,
#         2.38254244e+02,  2.37762838e+02,  2.35410700e+02,  2.32594736e+02,
#         2.30885366e+02,  2.31040273e+02,  2.32631113e+02,  2.31926204e+02,
#         2.31231596e+02,  2.31065116e+02,  2.31265766e+02,  2.31707563e+02,
#         2.31466239e+02,  2.31360672e+02,  2.31350775e+02,  2.31480627e+02,
#         2.31585056e+02,  2.31885138e+02,  2.31868973e+02,  2.32221031e+02,
#         2.32313185e+02,  2.32178173e+02,  2.32255829e+02,  2.32486656e+02,
#         2.32851712e+02,  2.32552307e+02,  2.32899059e+02,  2.32980195e+02,
#         2.33151912e+02,  2.33584665e+02,  2.32193000e+02,  2.31269269e+02,
#         2.31258875e+02,  2.31186554e+02,  2.31672788e+02,  2.30913368e+02,
#         2.30466096e+02,  2.30696926e+02,  2.31034969e+02,  2.31301614e+02,
#         2.31451981e+02,  2.31689179e+02,  2.31845779e+02,  2.32377956e+02,
#         2.32464443e+02,  2.31596040e+02,  2.30653335e+02,  2.28887028e+02,
#         2.28565083e+02,  2.29033714e+02,  2.29678573e+02,  2.30216706e+02,
#         2.30343459e+02,  2.30661856e+02,  2.31293100e+02,  2.32667003e+02,
#         2.32819993e+02,  2.32814247e+02,  2.32658914e+02,  2.32683911e+02,
#         2.33407616e+02,  2.35298005e+02,  2.35924831e+02,  2.35897835e+02,
#         2.35369204e+02,  2.34345377e+02,  2.34345851e+02,  2.34149745e+02,
#         2.34146339e+02,  2.34256445e+02,  2.34692881e+02,  2.35331709e+02,
#         2.35225301e+02,  2.34339133e+02,  2.33637498e+02,  2.33253592e+02,
#         2.33441994e+02,  2.33927953e+02,  2.31577941e+02,  2.29518196e+02,
#         2.29072676e+02,  2.29619670e+02,  2.30293231e+02,  2.30963856e+02,
#         2.31209487e+02,  2.31934863e+02,  2.32218985e+02,  2.31391827e+02,
#         2.29616886e+02,  2.28649646e+02,  2.26196650e+02,  2.25560517e+02,
#         2.25795042e+02,  2.26569287e+02,  2.28030153e+02,  2.28737323e+02,
#         2.29358124e+02,  2.29590645e+02,  2.29701254e+02,  2.28327506e+02,
#         2.27953630e+02,  2.31121338e+02,  2.33438451e+02,  2.36028236e+02,
#         2.38861639e+02,  2.45324390e+02,  2.54181906e+02,  2.57874406e+02,
#         2.57191287e+02,  2.55372504e+02,  2.56719173e+02,  2.54321264e+02,
#         2.52648515e+02,  2.50839968e+02,  2.49379085e+02,  2.51220934e+02,
#         2.52200858e+02,  2.54827342e+02,  2.56618533e+02,  2.57395331e+02,
#         2.62525732e+02,  2.61766763e+02,  2.60577504e+02,  2.59049304e+02,
#         2.60460752e+02,  2.66897937e+02,  2.70165545e+02,  2.75135453e+02,
#         2.74442350e+02,  2.71821860e+02,  2.69498208e+02,  2.69880800e+02,
#         2.71927528e+02,  2.68805819e+02,  2.67490238e+02,  2.66790512e+02,
#         2.69385779e+02,  2.69661551e+02,  2.68372276e+02,  2.65319007e+02,
#         2.65263273e+02,  2.63016111e+02,  2.61137577e+02,  2.59301472e+02,
#         2.60351471e+02,  2.61783015e+02,  2.58086055e+02,  2.56721813e+02,
#         2.55621166e+02,  2.56571141e+02,  2.57117310e+02,  2.55198833e+02,
#         2.52580851e+02,  2.52235893e+02,  2.53434394e+02,  2.55970801e+02,
#         2.56757838e+02,  2.62172836e+02,  2.65459610e+02,  2.68569041e+02,
#         2.66287919e+02,  2.58751355e+02,  2.53075088e+02,  2.52158517e+02,
#         2.55372793e+02,  2.56743788e+02,  2.55047182e+02,  2.52635792e+02,
#         2.52551120e+02,  2.56711343e+02,  2.60308818e+02,  2.58729859e+02,
#         2.56679715e+02,  2.56972728e+02,  2.69372107e+02,  2.81120413e+02,
#         2.87740022e+02,  2.86308784e+02,  2.77248983e+02,  2.74197614e+02,
#         2.75498261e+02,  2.75458036e+02,  2.76096129e+02,  2.75812954e+02,
#         2.76053701e+02,  2.78093577e+02,  2.77803102e+02,  2.75597980e+02,
#         2.73494341e+02,  2.71578953e+02,  2.71813205e+02,  2.71256693e+02,
#         2.70734673e+02,  2.68699637e+02,  2.65392586e+02,  2.62536447e+02,
#         2.62509256e+02,  2.62602309e+02,  2.63320241e+02,  2.64420624e+02,
#         2.66133051e+02,  2.66778889e+02,  2.68932015e+02,  2.68945119e+02,
#         2.66599825e+02,  2.65509003e+02,  2.64204162e+02,  2.62823751e+02,
#         2.62908711e+02,  2.63279850e+02,  2.63295064e+02,  2.62994263e+02,
#         2.61928001e+02,  2.60435953e+02,  2.60571078e+02,  2.57665235e+02,
#         2.58140806e+02,  2.58747077e+02,  2.59406853e+02,  2.60713805e+02,
#         2.59018802e+02,  2.57571299e+02,  2.55953558e+02,  2.57551066e+02,
#         2.63265576e+02,  2.65580201e+02,  2.66112819e+02,  2.65908450e+02,
#         2.63453655e+02,  2.62851969e+02,  2.64119000e+02,  2.65441815e+02,
#         2.65265650e+02,  2.64573374e+02,  2.63733632e+02,  2.64446241e+02,
#         2.66350958e+02,  2.66434201e+02,  2.65876322e+02,  2.65588294e+02,
#         2.66492488e+02,  2.68798498e+02,  2.68650645e+02,  2.67897285e+02,
#         2.66854022e+02,  2.66728442e+02,  2.67460350e+02,  2.67091557e+02,
#         2.66798735e+02,  2.67125998e+02,  2.66873690e+02,  2.65269945e+02,
#         2.63435553e+02,  2.62054229e+02,  2.62093025e+02,  2.60304747e+02,
#         2.58645104e+02,  2.57102874e+02,  2.56986325e+02,  2.58290334e+02,
#         2.57981904e+02,  2.57402819e+02,  2.57028973e+02,  2.56776937e+02,
#         2.56900197e+02,  2.57075054e+02,  2.57858306e+02,  2.58373754e+02,
#         2.58747461e+02,  2.59250516e+02,  2.59287214e+02,  2.59766262e+02,
#         2.60894132e+02,  2.63014686e+02,  2.63494973e+02,  2.61252075e+02,
#         2.54024633e+02,  2.39662045e+02,  2.31652433e+02,  2.26875320e+02,
#         2.30416665e+02,  2.34031804e+02,  2.34558821e+02,  2.33822727e+02,
#         2.31899421e+02,  2.33248943e+02,  2.34349816e+02,  2.34538966e+02,
#         2.34590262e+02,  2.36397520e+02,  2.34776469e+02,  2.36072342e+02,
#         2.36401566e+02,  2.36311378e+02,  2.37264752e+02,  2.36942746e+02,
#         2.35513603e+02,  2.34670971e+02,  2.35452948e+02,  2.34930869e+02,
#         2.35798065e+02,  2.35842115e+02,  2.36075894e+02,  2.37772519e+02,
#         2.38791342e+02,  2.38691229e+02,  2.37832002e+02,  2.36874227e+02,
#         2.36664829e+02,  2.37765475e+02,  2.38615455e+02,  2.38770111e+02,
#         2.39607306e+02,  2.39296419e+02,  2.36327821e+02,  2.35126753e+02,
#         2.34473872e+02,  2.34537434e+02,  2.35712489e+02,  2.35553615e+02,
#         2.34649634e+02,  2.34722611e+02,  2.35313987e+02,  2.33325437e+02,
#         2.32661111e+02,  2.33181625e+02,  2.33715580e+02,  2.33644231e+02,
#         2.33374129e+02,  2.32151562e+02,  2.33282475e+02,  2.34676734e+02,
#         2.35388250e+02,  2.36877859e+02,  2.37289972e+02,  2.38103209e+02,
#         2.39319875e+02,  2.40859791e+02,  2.40850869e+02,  2.41215390e+02,
#         2.42310959e+02,  2.40764946e+02,  2.37963023e+02,  2.33421566e+02,
#         2.32937483e+02,  2.34675050e+02,  2.35450114e+02,  2.36317563e+02,
#         2.35757401e+02,  2.34980898e+02,  2.35221590e+02,  2.35486092e+02,
#         2.35816237e+02,  2.36539010e+02,  2.36720269e+02,  2.37188661e+02,
#         2.37853425e+02,  2.37364814e+02,  2.36987487e+02,  2.35639915e+02,
#         2.35475901e+02,  2.36517463e+02,  2.36429779e+02,  2.36451224e+02,
#         2.35970158e+02,  2.36195328e+02,  2.36402438e+02,  2.36589944e+02,
#         2.37623566e+02,  2.36895338e+02,  2.36082513e+02,  2.35712250e+02,
#         2.35189222e+02,  2.35625339e+02,  2.36976163e+02,  2.38983546e+02,
#         2.38463561e+02,  2.36456867e+02,  2.35286106e+02,  2.34841302e+02,
#         2.35809717e+02,  2.36485574e+02,  2.36289123e+02,  2.33145704e+02,
#         2.31963946e+02,  2.29399455e+02,  2.28705307e+02,  2.27346522e+02,
#         2.26606482e+02,  2.29394950e+02,  2.30836588e+02,  2.34552180e+02,
#         2.36216110e+02,  2.37823042e+02,  2.40348737e+02,  2.39904137e+02,
#         2.39062368e+02,  2.39185870e+02,  2.38618387e+02,  2.40634798e+02,
#         2.44616259e+02,  2.45846799e+02,  2.46470973e+02,  2.49771966e+02,
#         2.51578944e+02,  2.54163982e+02,  2.52953372e+02,  2.49643599e+02,
#         2.48852919e+02,  2.47652403e+02,  2.48834266e+02,  2.49518645e+02,
#         2.49093601e+02,  2.49913463e+02,  2.49130224e+02,  2.47923963e+02,
#         2.48418565e+02,  2.49842290e+02,  2.47346819e+02,  2.49308411e+02,
#         2.48510757e+02,  2.48077412e+02,  2.48781314e+02,  2.47233493e+02,
#         2.48591349e+02,  2.47312865e+02,  2.44022709e+02,  2.41476044e+02,
#         2.41379049e+02,  2.43469714e+02,  2.45414978e+02,  2.46015126e+02,
#         2.45256129e+02,  2.45458415e+02,  2.44784441e+02,  2.44687076e+02,
#         2.44026877e+02,  2.43067745e+02,  2.41545130e+02,  2.42780420e+02,
#         2.43139603e+02,  2.42419530e+02,  2.38486450e+02,  2.36712537e+02,
#         2.35890650e+02,  2.38689726e+02,  2.41022386e+02,  2.44071908e+02,
#         2.46393169e+02,  2.47642384e+02,  2.48515942e+02,  2.50527552e+02,
#         2.54627433e+02,  2.55513232e+02,  2.55613509e+02,  2.54199428e+02,
#         2.53730012e+02,  2.56136413e+02,  2.57289483e+02,  2.55852277e+02,
#         2.52962337e+02,  2.52077211e+02,  2.53975110e+02,  2.55069515e+02,
#         2.59224743e+02,  2.72222040e+02,  2.77830216e+02,  2.79011620e+02,
#         2.85525187e+02,  2.78731130e+02,  2.73287064e+02,  2.73448386e+02,
#         2.74661524e+02,  2.79031251e+02,  2.78119293e+02,  2.73532402e+02,
#         2.72067934e+02,  2.74344916e+02,  2.75585077e+02,  2.75318514e+02,
#         2.76018246e+02,  2.86931149e+02,  2.93194970e+02,  2.95705779e+02,
#         2.94349120e+02,  2.91764288e+02,  2.87983852e+02,  2.85874944e+02,
#         2.83862303e+02,  2.81765080e+02,  2.80632823e+02,  2.81814016e+02,
#         2.81249387e+02,  2.77066808e+02,  2.70286482e+02,  2.65455230e+02,
#         2.63762271e+02,  2.64423044e+02,  2.65927687e+02,  2.62490581e+02,
#         2.58743402e+02,  2.58061735e+02,  2.55023087e+02,  2.56382844e+02,
#         2.56691250e+02,  2.57379994e+02,  2.60109920e+02,  2.61346014e+02,
#         2.61845830e+02,  2.61019827e+02,  2.58750627e+02,  2.56307151e+02,
#         2.55547795e+02,  2.55081143e+02,  2.55532913e+02,  2.54274869e+02,
#         2.53430925e+02,  2.54219510e+02,  2.53115679e+02,  2.53699760e+02,
#         2.52899819e+02,  2.53333772e+02,  2.54508921e+02,  2.55076309e+02,
#         2.55640912e+02,  2.54741330e+02,  2.53476832e+02,  2.56161455e+02,
#         2.57573377e+02,  2.57983192e+02,  2.55920351e+02,  2.54419828e+02,
#         2.54793018e+02,  2.55480191e+02,  2.55523960e+02,  2.55493766e+02,
#         2.55013411e+02,  2.55580921e+02,  2.54312868e+02,  2.53810845e+02,
#         2.53711498e+02,  2.53906321e+02,  2.53670156e+02,  2.54045612e+02,
#         2.53157081e+02,  2.51119384e+02,  2.50289520e+02,  2.50153812e+02,
#         2.50402205e+02,  2.47921800e+02,  2.49079872e+02,  2.49849570e+02,
#         2.53297522e+02,  2.54315238e+02,  2.53670052e+02,  2.53318350e+02,
#         2.52137619e+02,  2.51720622e+02,  2.53175894e+02,  2.53982849e+02,
#         2.55230406e+02,  2.55774911e+02,  2.53571587e+02,  2.53334509e+02,
#         2.49841370e+02,  2.48750220e+02,  2.49110458e+02,  2.50078648e+02,
#         2.52082977e+02,  2.52271189e+02,  2.51689597e+02,  2.50737119e+02,
#         2.50281063e+02,  2.49919213e+02,  2.50075164e+02,  2.49506566e+02,
#         2.49509025e+02,  2.52130781e+02,  2.53043517e+02,  2.53007378e+02,
#         2.51916304e+02,  2.50612934e+02,  2.52369186e+02,  2.53719209e+02,
#         2.54178240e+02,  2.53337939e+02,  2.53599679e+02,  2.52591170e+02,
#         2.52841170e+02,  2.53192000e+02,  2.53540018e+02,  2.53567726e+02,
#         2.52490531e+02,  2.52562813e+02,  2.53517152e+02,  2.55119807e+02,
#         2.55768506e+02,  2.53043249e+02,  2.50951334e+02,  2.49234102e+02,
#         2.49215580e+02,  2.49624727e+02,  2.49485507e+02,  2.48376076e+02,
#         2.47641528e+02,  2.46435169e+02,  2.46534931e+02,  2.47111758e+02,
#         2.47993935e+02,  2.47263767e+02,  2.46613407e+02,  2.44718590e+02,
#         2.42867429e+02,  2.42135847e+02,  2.41922972e+02,  2.42951178e+02,
#         2.43097380e+02,  2.43373507e+02,  2.43707321e+02,  2.44146266e+02,
#         2.44825072e+02,  2.44464291e+02,  2.44282323e+02,  2.42837500e+02,
#         2.42810414e+02,  2.42072255e+02,  2.42301255e+02,  2.42744824e+02,
#         2.43288663e+02,  2.43858169e+02,  2.43591427e+02,  2.42977756e+02,
#         2.40734120e+02,  2.40457136e+02,  2.39781600e+02,  2.40389559e+02,
#         2.41588736e+02,  2.41910137e+02,  2.39733614e+02,  2.37066908e+02,
#         2.35679285e+02,  2.33967137e+02,  2.36205378e+02,  2.36135378e+02,
#         2.36925052e+02,  2.37488757e+02,  2.36993390e+02,  2.36520496e+02,
#         2.35219938e+02,  2.34200307e+02,  2.34233050e+02,  2.34966342e+02,
#         2.36485315e+02,  2.36912269e+02,  2.37368022e+02,  2.37268644e+02,
#         2.37197767e+02,  2.37144129e+02,  2.36824683e+02,  2.36359653e+02,
#         2.35079115e+02,  2.34198419e+02,  2.29541726e+02,  2.30344358e+02,
#         2.32215121e+02,  2.35336682e+02,  2.37043579e+02,  2.36490359e+02,
#         2.35829754e+02,  2.36281985e+02,  2.35968848e+02,  2.37087085e+02,
#         2.36807413e+02,  2.37033837e+02,  2.36947974e+02,  2.36361232e+02,
#         2.36418724e+02,  2.36453115e+02,  2.36082445e+02,  2.35844375e+02,
#         2.35615006e+02,  2.36142494e+02,  2.36422227e+02,  2.37576630e+02,
#         2.37892488e+02,  2.38481234e+02,  2.38914159e+02,  2.39658290e+02,
#         2.39439896e+02,  2.40006925e+02,  2.40618128e+02,  2.41142491e+02,
#         2.41499472e+02,  2.40823835e+02,  2.40453402e+02,  2.39071401e+02,
#         2.39567023e+02,  2.39129248e+02,  2.39582406e+02,  2.39914149e+02,
#         2.39412721e+02,  2.39728692e+02,  2.39791558e+02,  2.41232991e+02,
#         2.41006940e+02,  2.40761599e+02,  2.40137490e+02,  2.39441378e+02,
#         2.39660956e+02,  2.38524283e+02,  2.39205036e+02,  2.38958385e+02,
#         2.37842964e+02,  2.37465859e+02,  2.35031336e+02,  2.34373494e+02,
#         2.34466682e+02,  2.33973491e+02,  2.35438215e+02,  2.35985103e+02,
#         2.35387914e+02,  2.33725760e+02,  2.35291549e+02,  2.35256077e+02,
#         2.35316354e+02,  2.33652743e+02,  2.32145690e+02,  2.32899735e+02,
#         2.34307567e+02,  2.34890399e+02,  2.35387266e+02,  2.35114819e+02,
#         2.34564707e+02,  2.35079603e+02,  2.34843467e+02,  2.34671478e+02,
#         2.34990541e+02,  2.36073418e+02,  2.35737464e+02,  2.36301900e+02,
#         2.35137704e+02,  2.34750262e+02,  2.34717914e+02,  2.34391901e+02,
#         2.33270217e+02,  2.31078497e+02,  2.26705456e+02,  2.23588628e+02,
#         2.22573043e+02,  2.23386174e+02,  2.22810765e+02,  2.23359518e+02,
#         2.25276160e+02,  2.27469957e+02,  2.28222089e+02,  2.26124751e+02,
#         2.24336640e+02,  2.23364133e+02,  2.25313555e+02,  2.28240229e+02,
#         2.29767744e+02,  2.31335160e+02,  2.32740891e+02,  2.34209181e+02,
#         2.33327418e+02,  2.32130642e+02,  2.31767621e+02,  2.31427866e+02,
#         2.33536132e+02,  2.35413113e+02,  2.36317401e+02,  2.37842820e+02,
#         2.37114661e+02,  2.36144132e+02,  2.35188523e+02,  2.34316882e+02,
#         2.34750352e+02,  2.35995594e+02,  2.36846681e+02,  2.36756980e+02,
#         2.36066446e+02,  2.35248782e+02,  2.35057865e+02,  2.35374255e+02,
#         2.36203337e+02,  2.36083358e+02,  2.35509024e+02,  2.35021974e+02,
#         2.34909753e+02,  2.35213662e+02,  2.35170660e+02,  2.35918348e+02,
#         2.36081900e+02,  2.36455816e+02,  2.36287434e+02,  2.36330687e+02,
#         2.36567633e+02,  2.36086652e+02,  2.35949970e+02,  2.36832336e+02,
#         2.36070093e+02,  2.36090654e+02,  2.36704063e+02,  2.36329030e+02,
#         2.37163917e+02,  2.37254182e+02,  2.37664168e+02,  2.37908189e+02,
#         2.37176863e+02,  2.36291402e+02,  2.35519482e+02,  2.36002813e+02,
#         2.36147054e+02,  2.36151687e+02,  2.35901203e+02,  2.35189199e+02,
#         2.34981476e+02,  2.35013209e+02,  2.35174866e+02,  2.36782029e+02,
#         2.38998549e+02,  2.39799012e+02,  2.40257000e+02,  2.40571534e+02,
#         2.40212782e+02,  2.41025175e+02,  2.41454308e+02,  2.41912521e+02,
#         2.42227121e+02,  2.41433243e+02,  2.41076260e+02,  2.41704214e+02,
#         2.41948431e+02,  2.42977101e+02,  2.43182937e+02,  2.43540693e+02,
#         2.43425192e+02,  2.42891534e+02,  2.41389610e+02,  2.40687607e+02,
#         2.40085058e+02,  2.39459084e+02,  2.39949687e+02,  2.39680681e+02,
#         2.39828562e+02,  2.40032786e+02,  2.39700333e+02,  2.42110907e+02,
#         2.43398338e+02,  2.44039072e+02,  2.44332022e+02,  2.43672299e+02]
# right_direction = 503
# wrong_direction = 497
# capital = 46.42494409212256
# +174
# -826

In [359]:
pred_result_pd = pd.Series(pred_result)

In [360]:
real_values = DF.iloc[262:len(pred_result)+263]

In [361]:
real_values

,<TICKER>,<PER>,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
262,XLTCZUSD,60,231011,230000,61.49,61.58,61.35,61.40,253
263,XLTCZUSD,60,231012,0,61.42,61.51,61.35,61.51,490
264,XLTCZUSD,60,231012,10000,61.51,61.51,61.17,61.36,513
265,XLTCZUSD,60,231012,20000,61.36,61.42,61.26,61.38,239
266,XLTCZUSD,60,231012,30000,61.41,61.53,61.31,61.53,796
...,...,...,...,...,...,...,...,...,...
758,XLTCZUSD,60,231101,150000,67.66,68.03,67.63,68.00,673
759,XLTCZUSD,60,231101,160000,68.01,68.62,67.64,68.40,671
760,XLTCZUSD,60,231101,170000,68.28,68.81,67.16,67.72,4858
761,XLTCZUSD,60,231101,180000,67.75,67.94,66.47,66.89,6920


In [362]:
right_direction = 0
wrong_direction = 0


In [363]:
for i in range(len(pred_result)):
  if (real_values['<CLOSE>'].iloc[i+1] > real_values['<CLOSE>'].iloc[i]):
    if (pred_result_pd.iloc[i] > real_values['<CLOSE>'].iloc[i]):
      right_direction += 1
    else:
      wrong_direction += 1
  else:
    if (real_values['<CLOSE>'].iloc[i] > pred_result_pd.iloc[i]):
      right_direction += 1
    else:
      wrong_direction += 1

In [364]:
right_direction

271

In [365]:
wrong_direction

229

In [366]:
capital = 100
count = 0
count_min = 0
for i in range(len(pred_result)):
  if (pred_result_pd.iloc[i] - real_values['<CLOSE>'].iloc[i] > 0):
    stop = real_values['<OPEN>'].iloc[i+1] - (pred_result_pd.iloc[i] - real_values['<OPEN>'].iloc[i+1])/3

    if (stop > real_values['<LOW>'].iloc[i+1]):
      capital -= real_values['<OPEN>'].iloc[i+1] - stop
      count_min +=1

    elif (real_values['<HIGH>'].iloc[i+1] > pred_result_pd.iloc[i]):
      capital += pred_result_pd.iloc[i] - real_values['<OPEN>'].iloc[i+1]
      count +=1

    elif (real_values['<CLOSE>'].iloc[i+1] > real_values['<OPEN>'].iloc[i+1]):
      capital += real_values['<CLOSE>'].iloc[i+1] - real_values['<OPEN>'].iloc[i+1]
      count +=1

    else:
      capital -= real_values['<OPEN>'].iloc[i+1] - real_values['<CLOSE>'].iloc[i+1]
      count_min +=1
  else:
    take = real_values['<OPEN>'].iloc[i+1] + (real_values['<OPEN>'].iloc[i+1] - pred_result_pd.iloc[i])/3

    if (take < real_values['<HIGH>'].iloc[i+1]):
      capital -= take - real_values['<OPEN>'].iloc[i+1]
      count_min +=1

    elif (real_values['<LOW>'].iloc[i+1] < pred_result_pd.iloc[i]):
      capital += real_values['<OPEN>'].iloc[i+1] - pred_result_pd.iloc[i]
      count +=1

    elif (real_values['<CLOSE>'].iloc[i] > real_values['<OPEN>'].iloc[i+1]):
      capital -= real_values['<CLOSE>'].iloc[i] - real_values['<OPEN>'].iloc[i+1]
      count_min +=1

    else:
      capital += real_values['<OPEN>'].iloc[i+1] - real_values['<CLOSE>'].iloc[i]
      count +=1
print(capital)
print(count)
print(count_min)

97.14308763308235
103
397
